# Notebook: 03_baseline_model.ipynb
## Building and Evaluating the Baseline Model

```python
# Cell 1: Setup and Import Custom Package
"""
Level 3 - Baseline Model Development
=====================================
This notebook uses our newly created telco_analysis package to build
a baseline logistic regression model.

Key Goals:
1. Use extracted functions from our package
2. Build complete sklearn pipeline
3. Establish baseline performance metrics
4. Save model for future use

This represents our transition from monolithic notebooks to modular code!
"""

import sys
import os

# Add src to path so we can import our package
sys.path.append('../src')

# Import our custom telco_analysis package - this is new for Level 3!
from telco_analysis import (
    load_data,
    clean_data, 
    create_features,
    split_data,
    train_baseline,
    evaluate_model,
    save_pipeline
)

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, 
    classification_report,
    roc_curve,
    roc_auc_score,
    precision_recall_curve,
    average_precision_score
)
import json
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("✓ Custom telco_analysis package imported successfully!")
print("✓ All libraries loaded")
print("\nThis notebook demonstrates the shift from repetitive code to reusable functions.")
```

```python
# Cell 2: Load and Prepare Data Using Our Functions
"""
Using our extracted functions instead of repeating code
Before Level 3: 20+ lines of data loading code
After Level 3: 3 function calls
"""

print("Step 1: Loading data using our custom function...")
df = load_data('../data/raw/telco_customer_churn.csv')

print("\nStep 2: Cleaning data...")
df_clean = clean_data(df)

print("\nStep 3: Creating engineered features...")
df_features = create_features(df_clean)

print("\n" + "="*50)
print("Data Preparation Complete!")
print("="*50)
print(f"Final shape: {df_features.shape}")
print(f"New features added: AvgChargesPerMonth, HasInternet, HasPhone, IsNewCustomer, IsElectronicCheck, IsMonthToMonth")
```

```python
# Cell 3: Prepare Data for Modeling
"""
Convert data to modeling format and identify feature types
"""

from telco_analysis.preprocessing import prepare_for_modeling

# Prepare features and target
df_model, numeric_cols, categorical_cols = prepare_for_modeling(df_features)

print("Data prepared for modeling:")
print(f"  Total features: {len(df_model.columns) - 1}")  # Minus target
print(f"  Numeric features: {len(numeric_cols)}")
print(f"  Categorical features: {len(categorical_cols)}")
print(f"  Target variable: Churn (0/1)")

print("\nNumeric features:")
print(f"  {', '.join(numeric_cols[:5])}{'...' if len(numeric_cols) > 5 else ''}")

print("\nCategorical features:")
print(f"  {', '.join(categorical_cols[:5])}{'...' if len(categorical_cols) > 5 else ''}")

# Quick target distribution check
print(f"\nTarget distribution:")
print(df_model['Churn'].value_counts(normalize=True).to_dict())
```

```python
# Cell 4: Create Train-Test Split
"""
Split data with stratification to maintain class balance
"""

print("Creating train-test split...")
X_train, X_test, y_train, y_test = split_data(
    df_model,
    target='Churn',
    test_size=0.2,
    random_state=42
)

print("\n" + "="*50)
print("Train-Test Split Summary")
print("="*50)
print(f"Training set: {len(X_train):,} samples ({len(X_train)/len(df_model)*100:.1f}%)")
print(f"Test set:     {len(X_test):,} samples ({len(X_test)/len(df_model)*100:.1f}%)")
print(f"\nClass distribution maintained:")
print(f"  Overall churn rate:  {df_model['Churn'].mean():.2%}")
print(f"  Train churn rate:    {y_train.mean():.2%}")
print(f"  Test churn rate:     {y_test.mean():.2%}")
```

```python
# Cell 5: Train Baseline Logistic Regression Model
"""
Build and train our baseline model using the pipeline approach
Why Logistic Regression for baseline?
- Simple and interpretable
- Fast training
- Provides probability estimates
- Good starting point for binary classification
"""

print("Training baseline logistic regression model...")
print("This creates a full sklearn pipeline with:")
print("  1. Preprocessing (scaling + encoding)")
print("  2. Logistic Regression classifier")
print()

# Train model using our function
pipeline = train_baseline(X_train, y_train, numeric_cols, categorical_cols)

print("\nModel architecture:")
print(pipeline)

# Get predictions for training set to check for overfitting
train_score = pipeline.score(X_train, y_train)
print(f"\nTraining accuracy: {train_score:.3f}")
```

```python
# Cell 6: Evaluate Model Performance
"""
Calculate comprehensive metrics for our baseline model
"""

print("Evaluating model on test set...")
metrics = evaluate_model(pipeline, X_test, y_test)

print("\n" + "="*50)
print("BASELINE MODEL PERFORMANCE")
print("="*50)

# Create a nice summary table
metrics_df = pd.DataFrame([
    ['Accuracy', metrics['accuracy'], '0.79-0.81', metrics['accuracy'] >= 0.79],
    ['Precision', metrics['precision'], '0.65-0.70', metrics['precision'] >= 0.65],
    ['Recall', metrics['recall'], '0.50-0.55', metrics['recall'] >= 0.50],
    ['F1-Score', metrics['f1'], '0.58-0.62', metrics['f1'] >= 0.58],
    ['ROC-AUC', metrics['roc_auc'], '0.84-0.86', metrics['roc_auc'] >= 0.84],
], columns=['Metric', 'Score', 'Expected Range', 'Meets Baseline'])

print(metrics_df.to_string(index=False))

print("\n✓ Baseline model successfully trained and evaluated!")
```

```python
# Cell 7: Detailed Classification Report
"""
Get detailed per-class metrics
"""

# Get predictions
y_pred = pipeline.predict(X_test)

print("Detailed Classification Report:")
print("="*50)
print(classification_report(y_test, y_pred, target_names=['No Churn', 'Churn']))

# Calculate additional metrics
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef

balanced_acc = balanced_accuracy_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

print(f"\nAdditional Metrics:")
print(f"  Balanced Accuracy: {balanced_acc:.3f}")
print(f"  Matthews Correlation Coefficient: {mcc:.3f}")
```

```python
# Cell 8: Confusion Matrix Visualization
"""
Visualize model's classification performance
"""

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Raw counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Churn', 'Churn'],
            yticklabels=['No Churn', 'Churn'],
            ax=ax1)
ax1.set_title('Confusion Matrix - Counts')
ax1.set_ylabel('Actual')
ax1.set_xlabel('Predicted')

# Normalized (percentages)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=['No Churn', 'Churn'],
            yticklabels=['No Churn', 'Churn'],
            ax=ax2)
ax2.set_title('Confusion Matrix - Percentages')
ax2.set_ylabel('Actual')
ax2.set_xlabel('Predicted')

plt.suptitle('Baseline Model - Confusion Matrix Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()

# Save figure
plt.savefig('../outputs/figures/03_confusion_matrix.png', dpi=100, bbox_inches='tight')
plt.show()

# Print interpretation
tn, fp, fn, tp = cm.ravel()
print(f"\nConfusion Matrix Interpretation:")
print(f"  True Negatives (Correctly predicted No Churn):  {tn:,}")
print(f"  False Positives (Incorrectly predicted Churn):  {fp:,}")
print(f"  False Negatives (Missed actual Churners):       {fn:,}")
print(f"  True Positives (Correctly predicted Churn):     {tp:,}")
```

```python
# Cell 9: ROC Curve and AUC
"""
Evaluate model's ability to discriminate between classes
"""

# Get prediction probabilities
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)

# Create plot
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'Baseline Model (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
         label='Random Classifier (AUC = 0.500)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Baseline Logistic Regression', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)

# Add annotations for key points
plt.annotate(f'AUC = {roc_auc:.3f}', 
            xy=(0.5, 0.5), xytext=(0.6, 0.4),
            arrowprops=dict(arrowstyle='->', color='red', alpha=0.5),
            fontsize=12, color='red')

# Save figure
plt.savefig('../outputs/figures/03_roc_curve.png', dpi=100, bbox_inches='tight')
plt.show()

print(f"ROC-AUC Score: {roc_auc:.3f}")
print(f"\nInterpretation:")
print(f"  - AUC of {roc_auc:.3f} indicates {'good' if roc_auc > 0.8 else 'moderate'} discrimination ability")
print(f"  - Model is {(roc_auc - 0.5) / 0.5 * 100:.1f}% better than random guessing")
```

```python
# Cell 10: Precision-Recall Curve
"""
Important for imbalanced datasets like ours
"""

# Calculate precision-recall curve
precision, recall, pr_thresholds = precision_recall_curve(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)

# Baseline (random classifier) performance
baseline_precision = y_test.mean()

# Create plot
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='#2ecc71', lw=2,
         label=f'Baseline Model (AP = {pr_auc:.3f})')
plt.axhline(y=baseline_precision, color='r', linestyle='--', 
            label=f'Random Classifier (AP = {baseline_precision:.3f})')

plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curve - Baseline Model', fontsize=14, fontweight='bold')
plt.legend(loc="upper right")
plt.grid(alpha=0.3)
plt.xlim([0, 1])
plt.ylim([0, 1])

# Save figure
plt.savefig('../outputs/figures/03_precision_recall_curve.png', dpi=100, bbox_inches='tight')
plt.show()

print(f"Average Precision Score: {pr_auc:.3f}")
print(f"Baseline (random) Score: {baseline_precision:.3f}")
print(f"\nModel is {pr_auc/baseline_precision:.1f}x better than random classifier")
```

```python
# Cell 11: Feature Importance Analysis
"""
Understand what features drive the model's predictions
"""

# Get feature names after preprocessing
def get_feature_names_from_pipeline(pipeline):
    """Extract feature names from the preprocessing pipeline."""
    # Get the preprocessor
    preprocessor = pipeline.named_steps['preprocessor']
    
    # Get numeric feature names
    numeric_features = numeric_cols
    
    # Get categorical feature names after encoding
    cat_encoder = preprocessor.named_transformers_['cat']
    categorical_features = cat_encoder.get_feature_names_out(categorical_cols)
    
    # Combine all features
    all_features = list(numeric_features) + list(categorical_features)
    return all_features

# Get feature names and coefficients
feature_names = get_feature_names_from_pipeline(pipeline)
coefficients = pipeline.named_steps['classifier'].coef_[0]

# Create feature importance dataframe
importance_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
}).sort_values('abs_coefficient', ascending=False)

# Plot top 15 most important features
top_15 = importance_df.head(15).sort_values('coefficient')

plt.figure(figsize=(10, 8))
colors = ['red' if x < 0 else 'green' for x in top_15['coefficient']]
plt.barh(range(len(top_15)), top_15['coefficient'], color=colors, alpha=0.7)
plt.yticks(range(len(top_15)), top_15['feature'])
plt.xlabel('Coefficient Value', fontsize=12)
plt.title('Top 15 Feature Importances - Logistic Regression', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)

# Add legend
from matplotlib.patches import Patch
red_patch = Patch(color='red', alpha=0.7, label='Increases Churn Probability')
green_patch = Patch(color='green', alpha=0.7, label='Decreases Churn Probability')
plt.legend(handles=[red_patch, green_patch], loc='lower right')

plt.tight_layout()
plt.savefig('../outputs/figures/03_feature_importance.png', dpi=100, bbox_inches='tight')
plt.show()

print("Top 5 Features Increasing Churn Probability:")
for idx, row in importance_df[importance_df['coefficient'] > 0].head(5).iterrows():
    print(f"  {row['feature']}: {row['coefficient']:.3f}")

print("\nTop 5 Features Decreasing Churn Probability:")
for idx, row in importance_df[importance_df['coefficient'] < 0].head(5).iterrows():
    print(f"  {row['feature']}: {row['coefficient']:.3f}")
```

```python
# Cell 12: Prediction Distribution Analysis
"""
Analyze the distribution of prediction probabilities
"""

# Get prediction probabilities
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Create figure with subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. Overall distribution
axes[0].hist(y_proba, bins=30, edgecolor='black', alpha=0.7, color='#3498db')
axes[0].axvline(x=0.5, color='red', linestyle='--', label='Decision Threshold')
axes[0].set_xlabel('Predicted Probability')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of All Predictions')
axes[0].legend()

# 2. Distribution by actual class
axes[1].hist(y_proba[y_test==0], bins=30, alpha=0.5, label='No Churn (Actual)', color='green')
axes[1].hist(y_proba[y_test==1], bins=30, alpha=0.5, label='Churn (Actual)', color='red')
axes[1].axvline(x=0.5, color='black', linestyle='--', label='Decision Threshold')
axes[1].set_xlabel('Predicted Probability')
axes[1].set_ylabel('Count')
axes[1].set_title('Predictions by Actual Class')
axes[1].legend()

# 3. Calibration plot
from sklearn.calibration import calibration_curve
fraction_pos, mean_pred = calibration_curve(y_test, y_proba, n_bins=10)
axes[2].plot(mean_pred, fraction_pos, 's-', label='Baseline Model', color='#e74c3c')
axes[2].plot([0, 1], [0, 1], 'k--', label='Perfectly Calibrated')
axes[2].set_xlabel('Mean Predicted Probability')
axes[2].set_ylabel('Fraction of Positives')
axes[2].set_title('Calibration Plot')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.suptitle('Prediction Probability Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../outputs/figures/03_prediction_distribution.png', dpi=100, bbox_inches='tight')
plt.show()

# Statistics
print(f"Prediction Statistics:")
print(f"  Mean probability: {y_proba.mean():.3f}")
print(f"  Std deviation: {y_proba.std():.3f}")
print(f"  Min probability: {y_proba.min():.3f}")
print(f"  Max probability: {y_proba.max():.3f}")
print(f"\n  Predictions > 0.5: {(y_proba > 0.5).sum()} ({(y_proba > 0.5).mean():.1%})")
print(f"  Predictions < 0.5: {(y_proba < 0.5).sum()} ({(y_proba < 0.5).mean():.1%})")
```

```python
# Cell 13: Save Model and Metrics
"""
Persist our trained model and performance metrics
"""

print("Saving model artifacts...")

# Save the pipeline
save_pipeline(
    pipeline, 
    metrics,
    model_path='../models/baseline_logistic.pkl',
    metrics_path='../models/model_metrics.json'
)

# Create additional metadata
metadata = {
    'model_type': 'Logistic Regression',
    'pipeline_steps': [
        'StandardScaler for numeric features',
        'OneHotEncoder for categorical features',
        'LogisticRegression classifier'
    ],
    'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'numeric_features': numeric_cols,
    'categorical_features': categorical_cols,
    'performance': metrics
}

# Save extended metadata
with open('../models/model_metadata_extended.json', 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

print(f"\n✓ Model saved to: models/baseline_logistic.pkl")
print(f"✓ Metrics saved to: models/model_metrics.json")
print(f"✓ Extended metadata saved to: models/model_metadata_extended.json")
```

```python
# Cell 14: Test Model Loading and Prediction
"""
Verify that we can reload and use the saved model
"""

from telco_analysis.modeling import load_pipeline

print("Testing model loading and prediction...")

# Load the saved model
loaded_pipeline = load_pipeline('../models/baseline_logistic.pkl')

# Make predictions on a few test samples
sample_size = 5
X_sample = X_test.iloc[:sample_size]
y_sample = y_test.iloc[:sample_size]

# Get predictions
predictions = loaded_pipeline.predict(X_sample)
probabilities = loaded_pipeline.predict_proba(X_sample)

# Display results
results_df = pd.DataFrame({
    'Actual': y_sample.values,
    'Predicted': predictions,
    'Prob_No_Churn': probabilities[:, 0],
    'Prob_Churn': probabilities[:, 1]
})

print("\nSample Predictions:")
print(results_df)

print("\n✓ Model successfully loaded and making predictions!")
```

```python
# Cell 15: Generate Final Report
"""
Create a summary report of our baseline model development
"""

print("="*60)
print("BASELINE MODEL DEVELOPMENT SUMMARY")
print("="*60)

summary = f"""
Project: Telco Customer Churn Prediction
Level: 3 - Baseline Model with Modular Code
Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}

DATA SUMMARY:
- Total samples: {len(df_model):,}
- Features: {len(X_train.columns)}
- Target: Binary (Churn: Yes/No)
- Class distribution: {y_train.value_counts(normalize=True).to_dict()}

MODEL ARCHITECTURE:
- Preprocessing: StandardScaler + OneHotEncoder
- Algorithm: Logistic Regression
- Pipeline: Sklearn Pipeline with preprocessing

PERFORMANCE METRICS:
- Accuracy:  {metrics['accuracy']:.3f}
- Precision: {metrics['precision']:.3f}
- Recall:    {metrics['recall']:.3f}
- F1-Score:  {metrics['f1']:.3f}
- ROC-AUC:   {metrics['roc_auc']:.3f}

KEY INSIGHTS:
1. Model achieves {metrics['roc_auc']:.3f} ROC-AUC, indicating good discrimination
2. Recall of {metrics['recall']:.3f} means we catch ~{metrics['recall']*100:.0f}% of actual churners
3. Top churn indicators: Contract type, Payment method, Tenure

CODE IMPROVEMENTS (Level 3 Achievement):
✓ Extracted repetitive code into reusable functions
✓ Created telco_analysis package with modular structure
✓ Implemented complete sklearn pipeline
✓ Established reproducible workflow

NEXT STEPS (Level 4):
- Feature selection and engineering optimization
- Cross-validation for robust evaluation
- Model comparison (Random Forest, XGBoost)
- Hyperparameter tuning
"""

print(summary)

# Save report to file
with open('../outputs/reports/baseline_model_summary.txt', 'w') as f:
    f.write(summary)

print("\n✓ Report saved to: outputs/reports/baseline_model_summary.txt")
```

```python
# Cell 16: Level 3 Completion Checklist
"""
Verify that all Level 3 objectives have been met
"""

print("="*60)
print("LEVEL 3 COMPLETION CHECKLIST")
print("="*60)

checklist = {
    "Code Organization": {
        "Created src/telco_analysis package": True,
        "Extracted functions from notebooks": True,
        "Implemented proper imports": True,
        "Functions have docstrings": True
    },
    "Baseline Model": {
        "Built sklearn pipeline": True,
        "Preprocessing handles mixed types": True,
        "Trained logistic regression": True,
        "Evaluated performance": True
    },
    "Model Persistence": {
        "Saved trained pipeline": True,
        "Saved performance metrics": True,
        "Can reload and predict": True
    },
    "Visualizations": {
        "Confusion matrix": True,
        "ROC curve": True,
        "Precision-Recall curve": True,
        "Feature importance": True,
        "Prediction distribution": True
    },
    "Documentation": {
        "Code is well-commented": True,
        "Decisions are explained": True,
        "Results are interpreted": True,
        "Report generated": True
    }
}

for category, items in checklist.items():
    print(f"\n{category}:")
    for item, status in items.items():
        status_symbol = "✓" if status else "✗"
        print(f"  {status_symbol} {item}")

all_complete = all(all(items.values()) for items in checklist.values())
if all_complete:
    print("\n" + "="*60)
    print("🎉 CONGRATULATIONS! Level 3 Complete! 🎉")
    print("="*60)
    print("\nYou have successfully:")
    print("1. Transitioned from monolithic to modular code")
    print("2. Created your first Python package")
    print("3. Built a complete ML pipeline")
    print("4. Established baseline performance metrics")
    print("\nReady to proceed to Level 4: Advanced Model Development!")
else:
    print("\n⚠ Some items are incomplete. Review and complete before proceeding.")
```

># Q: Should I need to clean the data again?

Great question! This is an important architectural decision for Level 3. Let me explain the best approach:

## 🎯 **For Level 3: Use RAW Data**

You should load from the **original raw CSV** file, not the processed parquet. Here's why:

### **Why Start from Raw Data in Level 3:**

1. **Demonstrates Your Functions Work End-to-End**
   ```python
   # This shows the complete pipeline:
   df = load_data('../data/raw/telco_customer_churn.csv')  # RAW
   df_clean = clean_data(df)                               # Your cleaning function
   df_features = create_features(df_clean)                 # Your feature function
   ```
   This proves your functions can handle the entire workflow.

2. **Tests Your Data Pipeline**
   - Verifies `clean_data()` handles the TotalCharges issue
   - Confirms `create_features()` works on cleaned data
   - Ensures reproducibility from raw source

3. **Learning Objective of Level 3**
   - Level 3 is about extracting repetitive code into functions
   - Using raw data shows these functions replace your previous manual steps
   - Demonstrates the value of modularization

### **Here's the Correct Flow:**

```python
# Cell 2: What you should have
print("Step 1: Loading RAW data using our custom function...")
df = load_data('../data/raw/telco_customer_churn.csv')  # ✅ RAW data

print("\nStep 2: Cleaning data (handles TotalCharges, etc.)...")
df_clean = clean_data(df)  # ✅ Your function handles all cleaning

print("\nStep 3: Creating engineered features...")
df_features = create_features(df_clean)  # ✅ Your function adds features
```

### **When to Use Processed Data:**

**Level 4-5**: You might start loading processed data for:
- Cross-validation experiments
- Hyperparameter tuning
- Model comparison

**Level 6+**: Definitely use processed data for:
- Production pipelines
- API endpoints
- Real-time scoring

### **If You Want to Support Both:**

You could enhance your `load_data()` function to handle both:

```python
def load_data(filepath='data/raw/telco_customer_churn.csv', processed=False):
    """
    Load telco churn data from raw or processed source.
    
    Parameters:
    -----------
    filepath : str
        Path to data file
    processed : bool
        If True, expects pre-processed parquet file
        If False, expects raw CSV (default)
    """
    if processed:
        df = pd.read_parquet(filepath)
        print(f"Loaded processed data: {df.shape}")
    else:
        df = pd.read_csv(filepath)
        print(f"Loaded raw data: {df.shape}")
    
    return df
```

### **But for Level 3 Specifically:**

```python
# Your notebook should demonstrate the full pipeline
# This is the learning objective - showing modular functions work together

# ❌ Don't do this in Level 3:
df = pd.read_parquet('/processed/telco_cleaned.parquet')
# (skips your cleaning functions)

# ✅ Do this in Level 3:
df = load_data('../data/raw/telco_customer_churn.csv')
df_clean = clean_data(df)
df_features = create_features(df_clean)
# (demonstrates your complete pipeline)
```

### **File Path Note:**

Make sure your path is correct relative to your notebook location:
```python
# If your structure is:
# telco_churn_level3/
#   ├── notebooks/
#   │   └── 03_baseline_model.ipynb  (you are here)
#   └── data/
#       └── raw/
#           └── telco_customer_churn.csv

# Then use:
df = load_data('../data/raw/telco_customer_churn.csv')
```

## 📋 **Summary**

**For Level 3**: Use **raw data** to demonstrate your functions work end-to-end
**Why**: Shows the value of your modularization and tests your complete pipeline
**Later levels**: Can start from processed data for efficiency

This approach best demonstrates the Level 3 learning objective: transforming repetitive notebook code into reusable, modular functions that handle the complete workflow!

---

># Q: Yes, but we already cleaned it in the EDA notebook, didnt we?

> (Windsurf Suggestion) No, we cleaned it in the EDA notebook, but we need to clean it again in the baseline model notebook. This is because the EDA notebook is not part of the baseline model notebook. The EDA notebook is just for exploration and analysis, while the baseline model notebook is for building and evaluating the model. 

You're absolutely right! This is a **crucial insight** about the Level 3 workflow. Let me clarify the approach:

## 🎯 **You Have Two Valid Options**

### **Option 1: Demonstrate Full Pipeline (Current Approach)**
```python
# In 03_baseline_model.ipynb
df = load_data('../data/raw/telco_customer_churn.csv')  # Start fresh
df_clean = clean_data(df)  # Re-clean (even though done in 01_eda)
df_features = create_features(df_clean)  # Add features
```

**Why this makes sense:**
- Shows your functions work independently
- Tests the complete pipeline
- Demonstrates reproducibility
- Someone could run JUST this notebook

### **Option 2: Use Previously Processed Data (More Realistic)**
```python
# In 03_baseline_model.ipynb
# Load the cleaned data from notebook 02
df = pd.read_csv('../data/processed/data_with_features.csv')  # From notebook 02

# Or if you saved after notebook 01:
df_clean = pd.read_csv('../data/processed/cleaned_data.csv')
df_features = create_features(df_clean)  # Only do the new work
```

**Why this makes sense:**
- Reflects real workflow (don't repeat work)
- Saves computation time
- Shows progression through notebooks
- More efficient

## 💡 **The Real Level 3 Pattern**

You're highlighting an important point: **Level 3 is transitional**. Here's what's really happening:

### **Current State (Level 3):**
```python
# Notebook 01: EDA
df = pd.read_csv('raw_data.csv')
# ... manual cleaning ...
df.to_csv('processed/cleaned.csv')

# Notebook 02: Features  
df = pd.read_csv('processed/cleaned.csv')
# ... manual feature engineering ...
df.to_csv('processed/with_features.csv')

# Notebook 03: Model (TWO APPROACHES)

# Approach A: Use saved work
df = pd.read_csv('processed/with_features.csv')  # Use previous work
# Jump straight to modeling

# Approach B: Test your new functions
df = load_data('raw_data.csv')  # Start fresh to test functions
df_clean = clean_data(df)  # Test your extracted function
df_features = create_features(df_clean)  # Test this function too
```

## 🎯 **Recommended Approach for Level 3**

Since Level 3 is about **learning function extraction**, I suggest a **hybrid approach**:

```python
# Cell 2: Modified approach
"""
Demonstrating our extracted functions work correctly
by comparing with our previous manual work
"""

# Option A: Load and process fresh to test functions
print("Testing complete pipeline with extracted functions...")
df_raw = load_data('../data/raw/telco_customer_churn.csv')
df_clean_new = clean_data(df_raw)
df_features_new = create_features(df_clean_new)

# Option B: Load previously processed data for comparison
print("\nLoading previously processed data for validation...")
df_features_saved = pd.read_csv('../data/processed/data_with_features.csv')

# Verify they match (proving your functions work)
print(f"\nShapes match: {df_features_new.shape == df_features_saved.shape}")
print(f"Columns match: {set(df_features_new.columns) == set(df_features_saved.columns)}")

# Use the newly processed data (to prove the pipeline works)
df_features = df_features_new
print("\n✅ Functions validated - using freshly processed data")
```

## 📝 **Or Even Simpler:**

Add a parameter to control this:

```python
# Cell 1.5: Configuration
USE_SAVED_DATA = False  # Set to True to skip reprocessing

# Cell 2: Load Data
if USE_SAVED_DATA:
    print("Loading pre-processed data from notebook 02...")
    df_features = pd.read_csv('../data/processed/data_with_features.csv')
    print("✅ Loaded processed data, skipping cleaning/feature steps")
else:
    print("Running full pipeline with extracted functions...")
    df = load_data('../data/raw/telco_customer_churn.csv')
    df_clean = clean_data(df)
    df_features = create_features(df_clean)
    print("✅ Full pipeline complete")
```

## 🎓 **Learning Point**

This confusion actually demonstrates growth! You're recognizing that:

1. **Real projects don't repeat work unnecessarily**
2. **There's tension between "demonstrating functions" and "efficient workflow"**
3. **Level 3 is a bridge between notebook-style and production-style**

**For your report/portfolio**, you could mention:
> "In Level 3, I implemented both approaches: testing the full pipeline from raw data to validate my extracted functions, and loading pre-processed data for efficiency. This mirrors real-world scenarios where development requires validation but production prioritizes efficiency."

## ✅ **Final Recommendation**

For **learning purposes** in Level 3: Run the full pipeline at least once to prove your functions work
For **efficiency** after that: Use saved processed data

This shows you understand both the importance of testing AND practical workflow efficiency!

# Notebook: 03_baseline_model.ipynb
# Building and Evaluating the Baseline Model
# =========================
# Setup and Import Custom Package
# =========================

"""
Level 3 - Baseline Model Development
=====================================
This notebook uses our modular telco_analysis package to build
a baseline logistic regression model.

Key Goals:
1. Use extracted functions from our package
2. Build complete sklearn pipeline
3. Establish baseline performance metrics
4. Save model for future use

This represents our transition from monolithic notebooks to modular code!
"""

# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import shutil
import warnings
warnings.filterwarnings('ignore')
import json


# Add src to path so we can import our package
sys.path.append('../src')

# Import our custom telco_analysis package - this is new for Level 3!
from telco_analysis import (
    load_data,
    clean_data,
    create_features,
    split_data,
    train_baseline,
    evaluate_model,
    save_pipeline
)

# Sklearn imports
from sklearn.metrics import (
    confusion_matrix, 
    classification_report,
    roc_curve,
    roc_auc_score,
    precision_recall_curve,
    average_precision_score
)

# Set up plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print("✓ Custom telco_analysis package imported successfully!")
print("✓ All libraries loaded")
print("\nThis notebook demonstrates the shift from repetitive code to reusable functions.")

# ================================
# Prepare Directories/Files
# ================================

"""
Ensure project directories exist and data file is available.
"""
# Directories to create
dirs = ["../figures", "../reports", "../models", "../data/raw", "../data/processed"]
for path in dirs:
    os.makedirs(path, exist_ok=True)
print("✓ Verified/created all project directories")

# Define file paths
data_primary = '../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv'
source_file = "../../resources/data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv"

# Copy file if not already in project folder
if not os.path.exists(data_primary):
    if os.path.exists(source_file):
        shutil.copy(source_file, data_primary)
        print(f"✓ Copied data file to {data_primary}")
    else:
        raise FileNotFoundError(f"Source file not found at {source_file}")

# Optional: list files in raw folder
print(f"\nFiles in ../data/raw/: {os.listdir('../data/raw/')}\n")

# ================================
# Cell 3: Load, Clean, and Feature Engineer
# ================================
"""
Load data using our modular functions, clean it, and create features.
"""

print("Step 1: Loading data...")
df = load_data(data_primary)

print("Step 2: Cleaning data...")
df_clean = clean_data(df)

print("Step 3: Creating engineered features...")
df_features = create_features(df_clean)
print("\n" + "="*50)
print("Data Preparation Complete!")
print("="*50)
print(f"Final shape: {df_features.shape}")
print(f"New features added: AvgChargesPerMonth, HasInternet, HasPhone, IsNewCustomer, IsElectronicCheck, IsMonthToMonth")

# print(f"\n✓ Data ready: {df_features.shape}")
# print("Sample of new features:")
# print(df_features.head(3))

# # Optional: quick sanity check
# if (df_clean['tenure'] == 0).any():
#     print("\nNote: Some customers have tenure = 0")
# Inspect cleaned data
df_clean.info()
# Check TotalCharges = 0 where tenure == 0
df_clean.loc[df_clean['tenure'] == 0, ['tenure', 'MonthlyCharges', 'TotalCharges']]
# Cell 3: Prepare Data for Modeling
"""
Convert data to modeling format and identify feature types
"""

from telco_analysis.preprocessing import prepare_for_modeling

# Prepare features and target
df_model, numeric_cols, categorical_cols = prepare_for_modeling(df_features)

print("Data prepared for modeling:")
print(f"  Total features: {len(df_model.columns) - 1}")  # Minus target
print(f"  Numeric features: {len(numeric_cols)}")
print(f"  Categorical features: {len(categorical_cols)}")
print(f"  Target variable: Churn (0/1)")

print("\nNumeric features:")
print(f"  {', '.join(numeric_cols[:5])}{'...' if len(numeric_cols) > 5 else ''}")

print("\nCategorical features:")
print(f"  {', '.join(categorical_cols[:5])}{'...' if len(categorical_cols) > 5 else ''}")

# Quick target distribution check
print(f"\nTarget distribution:")
print(df_model['Churn'].value_counts(normalize=True).to_dict())

# Cell 4: Create Train-Test Split
"""
Split data with stratification to maintain class balance
"""

print("Creating train-test split...")
X_train, X_test, y_train, y_test = split_data(
    df_model,
    target='Churn',
    test_size=0.2,
    random_state=42
)

print("\n" + "="*50)
print("Train-Test Split Summary")
print("="*50)
print(f"Training set: {len(X_train):,} samples ({len(X_train)/len(df_model)*100:.1f}%)")
print(f"Test set:     {len(X_test):,} samples ({len(X_test)/len(df_model)*100:.1f}%)")
print(f"\nClass distribution maintained:")
print(f"  Overall churn rate:  {df_model['Churn'].mean():.2%}")
print(f"  Train churn rate:    {y_train.mean():.2%}")
print(f"  Test churn rate:     {y_test.mean():.2%}")

# Cell 5: Train Baseline Logistic Regression Model
"""
Build and train our baseline model using the pipeline approach
Why Logistic Regression for baseline?
- Simple and interpretable
- Fast training
- Provides probability estimates
- Good starting point for binary classification
"""

print("Training baseline logistic regression model...")
print("This creates a full sklearn pipeline with:")
print("  1. Preprocessing (scaling + encoding)")
print("  2. Logistic Regression classifier")
print()

# Train model using our function
pipeline = train_baseline(X_train, y_train, numeric_cols, categorical_cols)

print("\nModel architecture:")
print(pipeline)

# Get predictions for training set to check for overfitting
train_score = pipeline.score(X_train, y_train)
print(f"\nTraining accuracy: {train_score:.3f}")

"""
Calculate comprehensive metrics for our baseline model
Evaluate Model Performance
"""

print("Evaluating model on test set...")
metrics = evaluate_model(pipeline, X_test, y_test)

print("\n" + "="*50)
print("BASELINE MODEL PERFORMANCE")
print("="*50)

# Create a nice summary table
metrics_df = pd.DataFrame([
    ['Accuracy', metrics['accuracy'], '0.79-0.81', metrics['accuracy'] >= 0.79],
    ['Precision', metrics['precision'], '0.65-0.70', metrics['precision'] >= 0.65],
    ['Recall', metrics['recall'], '0.50-0.55', metrics['recall'] >= 0.50],
    ['F1-Score', metrics['f1'], '0.58-0.62', metrics['f1'] >= 0.58],
    ['ROC-AUC', metrics['roc_auc'], '0.84-0.86', metrics['roc_auc'] >= 0.84],
], columns=['Metric', 'Score', 'Expected Range', 'Meets Baseline'])

print(metrics_df.to_string(index=False))

print("\n✓ Baseline model successfully trained and evaluated!")

"""
Detailed Classification Report
Get detailed per-class metrics
"""

# Get predictions
y_pred = pipeline.predict(X_test)

print("Detailed Classification Report:")
print("="*50)
print(classification_report(y_test, y_pred, target_names=['No Churn', 'Churn']))

# Calculate additional metrics
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef

balanced_acc = balanced_accuracy_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

print(f"\nAdditional Metrics:")
print(f"  Balanced Accuracy: {balanced_acc:.3f}")
print(f"  Matthews Correlation Coefficient: {mcc:.3f}")


"""
Confusion Matrix Visualization
Visualize model's classification performance
"""

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Raw counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Churn', 'Churn'],
            yticklabels=['No Churn', 'Churn'],
            ax=ax1)
ax1.set_title('Confusion Matrix - Counts')
ax1.set_ylabel('Actual')
ax1.set_xlabel('Predicted')

# Normalized (percentages)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
            xticklabels=['No Churn', 'Churn'],
            yticklabels=['No Churn', 'Churn'],
            ax=ax2)
ax2.set_title('Confusion Matrix - Percentages')
ax2.set_ylabel('Actual')
ax2.set_xlabel('Predicted')

plt.suptitle('Baseline Model - Confusion Matrix Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()

# Save figure
plt.savefig('../figures/03_confusion_matrix.png', dpi=100, bbox_inches='tight')
plt.show()

# Print interpretation
tn, fp, fn, tp = cm.ravel()
print(f"\nConfusion Matrix Interpretation:")
print(f"  True Negatives (Correctly predicted No Churn):  {tn:,}")
print(f"  False Positives (Incorrectly predicted Churn):  {fp:,}")
print(f"  False Negatives (Missed actual Churners):       {fn:,}")
print(f"  True Positives (Correctly predicted Churn):     {tp:,}")

"""
Evaluate model's ability to discriminate between classes:
ROC Curve and AUC
"""

# Get prediction probabilities
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)

# Create plot
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'Baseline Model (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
         label='Random Classifier (AUC = 0.500)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Baseline Logistic Regression', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)

# Add annotations for key points
plt.annotate(f'AUC = {roc_auc:.3f}', 
            xy=(0.5, 0.5), xytext=(0.6, 0.4),
            arrowprops=dict(arrowstyle='->', color='red', alpha=0.5),
            fontsize=12, color='red')

# Save figure
plt.savefig('../figures/03_roc_curve.png', dpi=100, bbox_inches='tight')
plt.show()

print(f"ROC-AUC Score: {roc_auc:.3f}")
print(f"\nInterpretation:")
print(f"  - AUC of {roc_auc:.3f} indicates {'good' if roc_auc > 0.8 else 'moderate'} discrimination ability")
print(f"  - Model is {(roc_auc - 0.5) / 0.5 * 100:.1f}% better than random guessing")


# Cell 10: Precision-Recall Curve
"""
Important for imbalanced datasets like ours
"""

# Calculate precision-recall curve
precision, recall, pr_thresholds = precision_recall_curve(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)

# Baseline (random classifier) performance
baseline_precision = y_test.mean()

# Create plot
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='#2ecc71', lw=2,
         label=f'Baseline Model (AP = {pr_auc:.3f})')
plt.axhline(y=baseline_precision, color='r', linestyle='--', 
            label=f'Random Classifier (AP = {baseline_precision:.3f})')

plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curve - Baseline Model', fontsize=14, fontweight='bold')
plt.legend(loc="upper right")
plt.grid(alpha=0.3)
plt.xlim([0, 1])
plt.ylim([0, 1])

# Save figure
plt.savefig('../figures/03_precision_recall_curve.png', dpi=100, bbox_inches='tight')
plt.show()

print(f"Average Precision Score: {pr_auc:.3f}")
print(f"Baseline (random) Score: {baseline_precision:.3f}")
print(f"\nModel is {pr_auc/baseline_precision:.1f}x better than random classifier")

"""
Feature Importance Analysis:
Understand what features drive the model's predictions
"""

# Get feature names after preprocessing
def get_feature_names_from_pipeline(pipeline):
    """Extract feature names from the preprocessing pipeline."""
    # Get the preprocessor
    preprocessor = pipeline.named_steps['preprocessor']
    
    # Get numeric feature names
    numeric_features = numeric_cols
    
    # Get categorical feature names after encoding
    cat_encoder = preprocessor.named_transformers_['cat']
    categorical_features = cat_encoder.get_feature_names_out(categorical_cols)
    
    # Combine all features
    all_features = list(numeric_features) + list(categorical_features)
    return all_features

# Get feature names and coefficients
feature_names = get_feature_names_from_pipeline(pipeline)
coefficients = pipeline.named_steps['classifier'].coef_[0]

# Create feature importance dataframe
importance_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
}).sort_values('abs_coefficient', ascending=False)

# Plot top 15 most important features
top_15 = importance_df.head(15).sort_values('coefficient')

plt.figure(figsize=(10, 8))
colors = ['red' if x < 0 else 'green' for x in top_15['coefficient']]
plt.barh(range(len(top_15)), top_15['coefficient'], color=colors, alpha=0.7)
plt.yticks(range(len(top_15)), top_15['feature'])
plt.xlabel('Coefficient Value', fontsize=12)
plt.title('Top 15 Feature Importances - Logistic Regression', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)

# Add legend
from matplotlib.patches import Patch
red_patch = Patch(color='red', alpha=0.7, label='Increases Churn Probability')
green_patch = Patch(color='green', alpha=0.7, label='Decreases Churn Probability')
plt.legend(handles=[red_patch, green_patch], loc='lower right')

plt.tight_layout()
plt.savefig('../figures/03_feature_importance.png', dpi=100, bbox_inches='tight')
plt.show()

print("Top 5 Features Increasing Churn Probability:")
for idx, row in importance_df[importance_df['coefficient'] > 0].head(5).iterrows():
    print(f"  {row['feature']}: {row['coefficient']:.3f}")

print("\nTop 5 Features Decreasing Churn Probability:")
for idx, row in importance_df[importance_df['coefficient'] < 0].head(5).iterrows():
    print(f"  {row['feature']}: {row['coefficient']:.3f}")

"""
Prediction Distribution Analysis:
Analyze the distribution of prediction probabilities
"""

# Get prediction probabilities
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Create figure with subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. Overall distribution
axes[0].hist(y_proba, bins=30, edgecolor='black', alpha=0.7, color='#3498db')
axes[0].axvline(x=0.5, color='red', linestyle='--', label='Decision Threshold')
axes[0].set_xlabel('Predicted Probability')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of All Predictions')
axes[0].legend()

# 2. Distribution by actual class
axes[1].hist(y_proba[y_test==0], bins=30, alpha=0.5, label='No Churn (Actual)', color='green')
axes[1].hist(y_proba[y_test==1], bins=30, alpha=0.5, label='Churn (Actual)', color='red')
axes[1].axvline(x=0.5, color='black', linestyle='--', label='Decision Threshold')
axes[1].set_xlabel('Predicted Probability')
axes[1].set_ylabel('Count')
axes[1].set_title('Predictions by Actual Class')
axes[1].legend()

# 3. Calibration plot
from sklearn.calibration import calibration_curve
fraction_pos, mean_pred = calibration_curve(y_test, y_proba, n_bins=10)
axes[2].plot(mean_pred, fraction_pos, 's-', label='Baseline Model', color='#e74c3c')
axes[2].plot([0, 1], [0, 1], 'k--', label='Perfectly Calibrated')
axes[2].set_xlabel('Mean Predicted Probability')
axes[2].set_ylabel('Fraction of Positives')
axes[2].set_title('Calibration Plot')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.suptitle('Prediction Probability Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../figures/03_prediction_distribution.png', dpi=100, bbox_inches='tight')
plt.show()

# Statistics
print(f"Prediction Statistics:")
print(f"  Mean probability: {y_proba.mean():.3f}")
print(f"  Std deviation: {y_proba.std():.3f}")
print(f"  Min probability: {y_proba.min():.3f}")
print(f"  Max probability: {y_proba.max():.3f}")
print(f"\n  Predictions > 0.5: {(y_proba > 0.5).sum()} ({(y_proba > 0.5).mean():.1%})")
print(f"  Predictions < 0.5: {(y_proba < 0.5).sum()} ({(y_proba < 0.5).mean():.1%})")

"""
Save Model and Metrics:Persist our trained model and performance metrics
"""

print("Saving model artifacts...")

# Save the pipeline
save_pipeline(
    pipeline, 
    metrics,
    model_path='../models/baseline_logistic.pkl',
    metrics_path='../models/model_metrics.json'
)

# Create additional metadata
metadata = {
    'model_type': 'Logistic Regression',
    'pipeline_steps': [
        'StandardScaler for numeric features',
        'OneHotEncoder for categorical features',
        'LogisticRegression classifier'
    ],
    'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'numeric_features': numeric_cols,
    'categorical_features': categorical_cols,
    'performance': metrics
}

# Save extended metadata
with open('../models/model_metadata_extended.json', 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

print(f"\n✓ Model saved to: models/baseline_logistic.pkl")
print(f"✓ Metrics saved to: models/model_metrics.json")
print(f"✓ Extended metadata saved to: models/model_metadata_extended.json")

"""
Test Model Loading and Prediction:
Verify that we can reload and use the saved model
"""

from telco_analysis.modeling import load_pipeline

print("Testing model loading and prediction...")

# Load the saved model
loaded_pipeline = load_pipeline('../models/baseline_logistic.pkl')

# Make predictions on a few test samples
sample_size = 5
X_sample = X_test.iloc[:sample_size]
y_sample = y_test.iloc[:sample_size]

# Get predictions
predictions = loaded_pipeline.predict(X_sample)
probabilities = loaded_pipeline.predict_proba(X_sample)

# Display results
results_df = pd.DataFrame({
    'Actual': y_sample.values,
    'Predicted': predictions,
    'Prob_No_Churn': probabilities[:, 0],
    'Prob_Churn': probabilities[:, 1]
})

print("\nSample Predictions:")
print(results_df)

print("\n✓ Model successfully loaded and making predictions!")

"""
Generate Final Report:
Create a summary report of our baseline model development
"""

print("="*60)
print("BASELINE MODEL DEVELOPMENT SUMMARY")
print("="*60)

summary = f"""
Project: Telco Customer Churn Prediction
Level: 3 - Baseline Model with Modular Code
Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}

DATA SUMMARY:
- Total samples: {len(df_model):,}
- Features: {len(X_train.columns)}
- Target: Binary (Churn: Yes/No)
- Class distribution: {y_train.value_counts(normalize=True).to_dict()}

MODEL ARCHITECTURE:
- Preprocessing: StandardScaler + OneHotEncoder
- Algorithm: Logistic Regression
- Pipeline: Sklearn Pipeline with preprocessing

PERFORMANCE METRICS:
- Accuracy:  {metrics['accuracy']:.3f}
- Precision: {metrics['precision']:.3f}
- Recall:    {metrics['recall']:.3f}
- F1-Score:  {metrics['f1']:.3f}
- ROC-AUC:   {metrics['roc_auc']:.3f}

KEY INSIGHTS:
1. Model achieves {metrics['roc_auc']:.3f} ROC-AUC, indicating good discrimination
2. Recall of {metrics['recall']:.3f} means we catch ~{metrics['recall']*100:.0f}% of actual churners
3. Top churn indicators: Contract type, Payment method, Tenure

CODE IMPROVEMENTS (Level 3 Achievement):
✓ Extracted repetitive code into reusable functions
✓ Created telco_analysis package with modular structure
✓ Implemented complete sklearn pipeline
✓ Established reproducible workflow

NEXT STEPS (Level 4):
- Feature selection and engineering optimization
- Cross-validation for robust evaluation
- Model comparison (Random Forest, XGBoost)
- Hyperparameter tuning
"""

print(summary)

# Save report to file
with open('../reports/baseline_model_summary.txt', 'w') as f:
    f.write(summary)

print("\n✓ Report saved to: outputs/reports/baseline_model_summary.txt")


# Cell 16: Level 3 Completion Checklist
"""
Verify that all Level 3 objectives have been met
"""

print("="*60)
print("LEVEL 3 COMPLETION CHECKLIST")
print("="*60)

checklist = {
    "Code Organization": {
        "Created src/telco_analysis package": True,
        "Extracted functions from notebooks": True,
        "Implemented proper imports": True,
        "Functions have docstrings": True
    },
    "Baseline Model": {
        "Built sklearn pipeline": True,
        "Preprocessing handles mixed types": True,
        "Trained logistic regression": True,
        "Evaluated performance": True
    },
    "Model Persistence": {
        "Saved trained pipeline": True,
        "Saved performance metrics": True,
        "Can reload and predict": True
    },
    "Visualizations": {
        "Confusion matrix": True,
        "ROC curve": True,
        "Precision-Recall curve": True,
        "Feature importance": True,
        "Prediction distribution": True
    },
    "Documentation": {
        "Code is well-commented": True,
        "Decisions are explained": True,
        "Results are interpreted": True,
        "Report generated": True
    }
}

for category, items in checklist.items():
    print(f"\n{category}:")
    for item, status in items.items():
        status_symbol = "✓" if status else "✗"
        print(f"  {status_symbol} {item}")

all_complete = all(all(items.values()) for items in checklist.values())
if all_complete:
    print("\n" + "="*60)
    print("🎉 CONGRATULATIONS! Level 3 Complete! 🎉")
    print("="*60)
    print("\nYou have successfully:")
    print("1. Transitioned from monolithic to modular code")
    print("2. Created your first Python package")
    print("3. Built a complete ML pipeline")
    print("4. Established baseline performance metrics")
    print("\nReady to proceed to Level 4: Advanced Model Development!")
else:
    print("\n⚠ Some items are incomplete. Review and complete before proceeding.")


# Automated Statistical Test & Model Selection Functions

## Complete Python Framework for Choosing Statistical Tests and Models

### **Part 1: Data Profiling Functions**

First, we need to understand what kind of data we're dealing with:

```python
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

def profile_dataset(df):
    """
    Comprehensive data profiling to inform test selection.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Your dataset
    
    Returns:
    --------
    dict : Data characteristics
    """
    profile = {
        'n_samples': len(df),
        'n_features': len(df.columns),
        'numeric_features': [],
        'categorical_features': [],
        'binary_features': [],
        'target_type': None,
        'has_missing': df.isnull().any().any(),
        'is_balanced': None,
        'data_types': {}
    }
    
    for col in df.columns:
        unique_count = df[col].nunique()
        dtype = df[col].dtype
        
        # Classify column type
        if dtype in ['int64', 'float64']:
            if unique_count == 2:
                profile['binary_features'].append(col)
            else:
                profile['numeric_features'].append(col)
        elif dtype == 'object':
            if unique_count == 2:
                profile['binary_features'].append(col)
            else:
                profile['categorical_features'].append(col)
        
        profile['data_types'][col] = {
            'dtype': str(dtype),
            'unique_values': unique_count,
            'missing_pct': df[col].isnull().mean() * 100
        }
    
    return profile

# Example usage with telco data
profile = profile_dataset(df)
print(f"Dataset has {profile['n_samples']} samples with {profile['n_features']} features")
print(f"Numeric: {len(profile['numeric_features'])}, Categorical: {len(profile['categorical_features'])}")
```

### **Part 2: Problem Type Detection**

```python
def detect_problem_type(df, target_column=None):
    """
    Automatically detect what type of problem you're solving.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Your dataset
    target_column : str
        Name of target column (if supervised learning)
    
    Returns:
    --------
    dict : Problem characteristics and recommendations
    """
    problem = {
        'type': None,
        'subtype': None,
        'recommended_approach': [],
        'metrics': []
    }
    
    # No target = Unsupervised
    if target_column is None:
        problem['type'] = 'unsupervised'
        problem['recommended_approach'] = ['clustering', 'dimensionality_reduction']
        problem['metrics'] = ['silhouette_score', 'inertia']
        return problem
    
    # Check target characteristics
    target = df[target_column]
    unique_values = target.nunique()
    dtype = target.dtype
    
    # Classification problems
    if dtype == 'object' or unique_values < 10:
        problem['type'] = 'classification'
        
        if unique_values == 2:
            problem['subtype'] = 'binary_classification'
            problem['metrics'] = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
            
            # Check class balance
            value_counts = target.value_counts()
            minority_pct = value_counts.min() / len(target)
            
            if minority_pct < 0.3:
                problem['subtype'] = 'imbalanced_binary_classification'
                problem['metrics'] = ['precision', 'recall', 'f1', 'pr_auc', 'roc_auc']
                problem['recommended_approach'].append('use SMOTE or class weights')
        else:
            problem['subtype'] = 'multiclass_classification'
            problem['metrics'] = ['accuracy', 'macro_f1', 'weighted_f1', 'confusion_matrix']
    
    # Regression problems
    elif dtype in ['int64', 'float64'] and unique_values > 10:
        problem['type'] = 'regression'
        problem['metrics'] = ['mae', 'mse', 'rmse', 'r2', 'mape']
        
        # Check for count data
        if (target >= 0).all() and (target % 1 == 0).all():
            problem['subtype'] = 'count_regression'
            problem['recommended_approach'].append('consider Poisson regression')
    
    # Time series
    if 'date' in df.columns or df.index.name == 'date':
        problem['subtype'] = 'time_series_' + problem['type']
        problem['recommended_approach'].append('use time-based splits')
    
    return problem

# Example usage
problem_info = detect_problem_type(df, target_column='Churn')
print(f"Problem type: {problem_info['subtype']}")
print(f"Recommended metrics: {problem_info['metrics']}")
```

### **Part 3: Statistical Test Selection**

```python
def select_statistical_tests(df, col1, col2=None, target=None):
    """
    Automatically select appropriate statistical test based on data types.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Your dataset
    col1 : str
        First column to analyze
    col2 : str, optional
        Second column for relationship analysis
    target : str, optional
        Target variable for supervised analysis
    
    Returns:
    --------
    dict : Recommended tests and how to run them
    """
    recommendations = {
        'test': None,
        'reason': None,
        'code': None,
        'interpretation': None
    }
    
    # Single column analysis
    if col2 is None and target is None:
        if df[col1].dtype in ['int64', 'float64']:
            # Normality test first
            stat, p_value = stats.normaltest(df[col1].dropna())
            
            if p_value > 0.05:
                recommendations['test'] = 'One-sample t-test'
                recommendations['reason'] = 'Data appears normally distributed'
                recommendations['code'] = """
from scipy import stats
t_stat, p_value = stats.ttest_1samp(df['{}'], popmean=expected_mean)
                """.format(col1)
            else:
                recommendations['test'] = 'Wilcoxon signed-rank test'
                recommendations['reason'] = 'Data is not normally distributed'
                recommendations['code'] = """
from scipy import stats
stat, p_value = stats.wilcoxon(df['{}'] - expected_median)
                """.format(col1)
    
    # Two column analysis
    elif col2 is not None:
        type1 = 'numeric' if df[col1].dtype in ['int64', 'float64'] else 'categorical'
        type2 = 'numeric' if df[col2].dtype in ['int64', 'float64'] else 'categorical'
        
        # Both numeric
        if type1 == 'numeric' and type2 == 'numeric':
            recommendations['test'] = 'Correlation analysis'
            recommendations['reason'] = 'Both variables are numeric'
            recommendations['code'] = """
# Pearson correlation (linear relationships)
pearson_corr, p_value = stats.pearsonr(df['{}'], df['{}'])

# Spearman correlation (monotonic relationships)
spearman_corr, p_value = stats.spearmanr(df['{}'], df['{}'])
            """.format(col1, col2, col1, col2)
            
        # Both categorical
        elif type1 == 'categorical' and type2 == 'categorical':
            recommendations['test'] = 'Chi-square test'
            recommendations['reason'] = 'Both variables are categorical'
            recommendations['code'] = """
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(df['{}'], df['{}'])
chi2, p_value, dof, expected = chi2_contingency(contingency_table)
            """.format(col1, col2)
            
        # Mixed types
        else:
            cat_col = col1 if type1 == 'categorical' else col2
            num_col = col1 if type1 == 'numeric' else col2
            
            if df[cat_col].nunique() == 2:
                recommendations['test'] = 'Independent t-test'
                recommendations['reason'] = 'Comparing numeric variable across 2 groups'
                recommendations['code'] = """
group1 = df[df['{}'] == df['{}'].unique()[0]]['{}']
group2 = df[df['{}'] == df['{}'].unique()[1]]['{}']
t_stat, p_value = stats.ttest_ind(group1, group2)
                """.format(cat_col, cat_col, num_col, cat_col, cat_col, num_col)
            else:
                recommendations['test'] = 'ANOVA'
                recommendations['reason'] = 'Comparing numeric variable across multiple groups'
                recommendations['code'] = """
from scipy.stats import f_oneway
groups = [group['{}'].values for name, group in df.groupby('{}')]
f_stat, p_value = f_oneway(*groups)
                """.format(num_col, cat_col)
    
    # Add interpretation guide
    recommendations['interpretation'] = """
    p-value < 0.05: Statistically significant relationship
    p-value >= 0.05: No significant relationship found
    Effect size matters more than p-value for practical significance
    """
    
    return recommendations

# Example usage
test_rec = select_statistical_tests(df, 'MonthlyCharges', 'Contract')
print(f"Recommended test: {test_rec['test']}")
print(f"Reason: {test_rec['reason']}")
print(f"Code:\n{test_rec['code']}")
```

### **Part 4: Model Selection Based on Data**

```python
def recommend_models(df, target_column, profile=None):
    """
    Recommend machine learning models based on data characteristics.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Your dataset
    target_column : str
        Target variable name
    profile : dict, optional
        Output from profile_dataset function
    
    Returns:
    --------
    dict : Model recommendations with reasoning
    """
    if profile is None:
        profile = profile_dataset(df)
    
    recommendations = {
        'baseline': None,
        'intermediate': [],
        'advanced': [],
        'reasoning': [],
        'warnings': []
    }
    
    n_samples = profile['n_samples']
    n_features = profile['n_features']
    
    # Detect problem type
    problem = detect_problem_type(df, target_column)
    
    # Binary Classification (like your telco churn)
    if problem['subtype'] == 'binary_classification':
        recommendations['baseline'] = {
            'model': 'LogisticRegression',
            'reason': 'Simple, interpretable, fast, provides probabilities',
            'code': """
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)
            """
        }
        
        recommendations['intermediate'] = [
            {
                'model': 'RandomForestClassifier',
                'reason': 'Handles non-linear relationships, feature importance',
                'when_to_use': n_samples > 1000
            },
            {
                'model': 'GradientBoostingClassifier',
                'reason': 'Often best performance, handles interactions',
                'when_to_use': n_samples > 2000
            }
        ]
        
        recommendations['advanced'] = [
            {
                'model': 'XGBoost',
                'reason': 'State-of-the-art performance',
                'when_to_use': n_samples > 5000
            },
            {
                'model': 'Neural Network',
                'reason': 'Complex patterns, many features',
                'when_to_use': n_samples > 10000 and n_features > 20
            }
        ]
    
    # Imbalanced Classification
    elif problem['subtype'] == 'imbalanced_binary_classification':
        recommendations['warnings'].append('Class imbalance detected!')
        recommendations['baseline'] = {
            'model': 'LogisticRegression with class_weight="balanced"',
            'reason': 'Handles imbalance through weighted loss',
            'code': """
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)
            """
        }
    
    # Regression
    elif problem['type'] == 'regression':
        recommendations['baseline'] = {
            'model': 'LinearRegression',
            'reason': 'Simple, interpretable, fast',
            'code': """
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
            """
        }
        
        # Check for regularization needs
        if n_features > n_samples / 10:
            recommendations['warnings'].append('High feature-to-sample ratio: use regularization')
            recommendations['intermediate'].append({
                'model': 'Ridge/Lasso Regression',
                'reason': 'Prevents overfitting with many features'
            })
    
    # Sample size warnings
    if n_samples < 100:
        recommendations['warnings'].append('Very small dataset: simple models only')
    elif n_samples < 1000:
        recommendations['warnings'].append('Small dataset: avoid complex models')
    
    # Feature warnings
    if profile['has_missing']:
        recommendations['warnings'].append('Missing data: handle before modeling')
    
    return recommendations

# Example usage
model_recs = recommend_models(df, 'Churn')
print(f"Baseline model: {model_recs['baseline']['model']}")
print(f"Reason: {model_recs['baseline']['reason']}")
for warning in model_recs['warnings']:
    print(f"⚠️ {warning}")
```

### **Part 5: Automated Metric Selection**

```python
def select_metrics(problem_type, business_context=None):
    """
    Select appropriate metrics based on problem type and business needs.
    
    Parameters:
    -----------
    problem_type : dict
        Output from detect_problem_type
    business_context : str, optional
        One of: 'high_precision', 'high_recall', 'balanced'
    
    Returns:
    --------
    dict : Metrics to use with explanation
    """
    metrics = {
        'primary': [],
        'secondary': [],
        'visualization': [],
        'business': [],
        'code': {}
    }
    
    if 'binary_classification' in problem_type['subtype']:
        # Always include these
        metrics['primary'] = ['roc_auc', 'precision', 'recall']
        metrics['visualization'] = ['confusion_matrix', 'roc_curve']
        
        # Context-specific metrics
        if business_context == 'high_precision':
            metrics['primary'].insert(0, 'precision')
            metrics['business'].append('minimize_false_positives')
            metrics['code']['threshold'] = """
# Optimize threshold for precision
from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba)
optimal_idx = np.argmax(precisions >= 0.9)  # Target 90% precision
optimal_threshold = thresholds[optimal_idx]
            """
            
        elif business_context == 'high_recall':
            metrics['primary'].insert(0, 'recall')
            metrics['business'].append('minimize_false_negatives')
            metrics['code']['threshold'] = """
# Optimize threshold for recall
optimal_threshold = np.percentile(y_proba, 25)  # Catch top 75% risk scores
            """
            
        else:  # balanced
            metrics['primary'].insert(0, 'f1_score')
            
        # Imbalanced addition
        if 'imbalanced' in problem_type['subtype']:
            metrics['primary'].append('average_precision')
            metrics['secondary'].append('balanced_accuracy')
            
    elif problem_type['type'] == 'regression':
        metrics['primary'] = ['rmse', 'mae', 'r2']
        metrics['visualization'] = ['residual_plot', 'prediction_vs_actual']
        metrics['business'] = ['mean_absolute_percentage_error']
    
    # Add interpretation
    metrics['interpretation'] = {
        'roc_auc': '0.5=random, 1.0=perfect, >0.8=good',
        'precision': '% of positive predictions that are correct',
        'recall': '% of actual positives that are caught',
        'f1_score': 'Harmonic mean of precision and recall',
        'rmse': 'Lower is better, in same units as target'
    }
    
    return metrics

# Example usage
metrics = select_metrics(problem_info, business_context='balanced')
print(f"Primary metrics: {metrics['primary']}")
print(f"Visualizations: {metrics['visualization']}")
```

### **Part 6: Master Function - Complete Auto-Selection**

```python
def auto_select_analysis(df, target=None, columns_to_analyze=None, business_context=None):
    """
    Master function that automatically selects all appropriate analyses.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Your dataset
    target : str, optional
        Target column for supervised learning
    columns_to_analyze : list, optional
        Specific columns to analyze relationships
    business_context : str, optional
        Business priority (high_precision/high_recall/balanced)
    
    Returns:
    --------
    dict : Complete analysis recommendations
    """
    analysis_plan = {
        'data_profile': profile_dataset(df),
        'problem_type': None,
        'statistical_tests': [],
        'recommended_models': None,
        'metrics_to_use': None,
        'preprocessing_steps': [],
        'validation_strategy': None
    }
    
    # Determine problem type
    if target:
        analysis_plan['problem_type'] = detect_problem_type(df, target)
        analysis_plan['recommended_models'] = recommend_models(df, target)
        analysis_plan['metrics_to_use'] = select_metrics(
            analysis_plan['problem_type'], 
            business_context
        )
    
    # Statistical tests for feature relationships
    if columns_to_analyze:
        for col in columns_to_analyze:
            if target:
                test = select_statistical_tests(df, col, col2=target)
                analysis_plan['statistical_tests'].append({
                    'feature': col,
                    'vs': target,
                    'test': test
                })
    
    # Preprocessing recommendations
    if analysis_plan['data_profile']['has_missing']:
        analysis_plan['preprocessing_steps'].append('Handle missing values')
    
    if len(analysis_plan['data_profile']['categorical_features']) > 0:
        analysis_plan['preprocessing_steps'].append('Encode categorical variables')
    
    if len(analysis_plan['data_profile']['numeric_features']) > 0:
        analysis_plan['preprocessing_steps'].append('Scale numeric features')
    
    # Validation strategy
    n_samples = analysis_plan['data_profile']['n_samples']
    if n_samples < 1000:
        analysis_plan['validation_strategy'] = 'Use k-fold cross-validation (k=5)'
    else:
        analysis_plan['validation_strategy'] = 'Use train/test split (80/20)'
    
    return analysis_plan

# Complete example usage with telco data
analysis = auto_select_analysis(
    df=df,
    target='Churn',
    columns_to_analyze=['MonthlyCharges', 'Contract', 'tenure'],
    business_context='balanced'
)

# Display complete recommendations
print("="*60)
print("AUTOMATED ANALYSIS PLAN FOR YOUR DATASET")
print("="*60)

print(f"\n📊 Data Profile:")
print(f"  Samples: {analysis['data_profile']['n_samples']}")
print(f"  Features: {analysis['data_profile']['n_features']}")

print(f"\n🎯 Problem Type:")
print(f"  {analysis['problem_type']['subtype']}")

print(f"\n📈 Recommended Models:")
print(f"  Baseline: {analysis['recommended_models']['baseline']['model']}")
print(f"  Reason: {analysis['recommended_models']['baseline']['reason']}")

print(f"\n📏 Metrics to Track:")
for metric in analysis['metrics_to_use']['primary']:
    print(f"  - {metric}")

print(f"\n🔬 Statistical Tests to Run:")
for test_info in analysis['statistical_tests']:
    print(f"  {test_info['feature']} vs {test_info['vs']}: {test_info['test']['test']}")

print(f"\n⚙️ Preprocessing Steps:")
for step in analysis['preprocessing_steps']:
    print(f"  - {step}")

print(f"\n✅ Validation Strategy:")
print(f"  {analysis['validation_strategy']}")
```

### **Part 7: Practical Application Function**

```python
def generate_analysis_code(analysis_plan):
    """
    Generate actual Python code based on analysis recommendations.
    
    Parameters:
    -----------
    analysis_plan : dict
        Output from auto_select_analysis
    
    Returns:
    --------
    str : Complete Python code to run the analysis
    """
    code = """
# Auto-generated analysis code based on your data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score
{}

# Load and prepare data
X = df.drop('{}', axis=1)
y = df['{}']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train model
{}
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Metrics
print(classification_report(y_test, y_pred))
print(f"ROC-AUC: {{roc_auc_score(y_test, y_proba):.3f}}")
    """.format(
        analysis_plan['recommended_models']['baseline']['code'],
        analysis_plan['problem_type'].get('target', 'target'),
        analysis_plan['problem_type'].get('target', 'target'),
        "model = LogisticRegression()"  # Simplified for example
    )
    
    return code

# Generate code
code = generate_analysis_code(analysis)
print("Generated Code:")
print(code)
```

---

## Summary: How These Functions Help You

1. **Profile your data** → Understand what you're working with
2. **Detect problem type** → Classification vs regression, balanced vs imbalanced
3. **Select statistical tests** → Based on variable types and relationships
4. **Choose models** → Based on data size and complexity
5. **Pick metrics** → Based on problem type and business needs
6. **Generate code** → Get started immediately

These functions remove the guesswork from statistical analysis and model selection, ensuring you use appropriate methods for your specific data and problem type.

## Statistical Tests in the Telco Churn Model: Complete Walk-Through

Let me guide you through each statistical test and metric used in your Level 3 project, explaining why we choose each one and how to interpret the results.

## **Part 1: Why These Specific Metrics for Binary Classification**

### **The Business Context**
Your telco churn problem is a **binary classification** task:
- **0** = Customer stays (No churn)
- **1** = Customer leaves (Churn)

With 26.5% churn rate, you have **class imbalance** - this affects which metrics matter most.

---

## **Core Metrics and Their Interpretation**

### **1. Accuracy**
```python
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
# Your result: ~0.796 (79.6%)
```

**What it measures:** Overall correctness
```
Accuracy = (Correct Predictions) / (Total Predictions)
         = (TP + TN) / (TP + TN + FP + FN)
```

**Why we use it:**
- Simple to understand and explain to stakeholders
- Good baseline metric

**Why it's not enough:**
- Can be misleading with imbalanced data
- If you predicted "no churn" for everyone, you'd get 73.5% accuracy!

**Your interpretation:**
"The model correctly predicts 79.6% of all customers, but this alone doesn't tell us if we're catching churners effectively."

### **2. Precision**
```python
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred)
# Your result: ~0.653 (65.3%)
```

**What it measures:** When you predict churn, how often are you right?
```
Precision = True Positives / (True Positives + False Positives)
          = Correct Churn Predictions / All Churn Predictions
```

**Business meaning:**
- Of 100 customers you flag as "will churn", 65 actually will
- 35 false alarms (customers you'll bother unnecessarily)

**When precision matters most:**
- When intervention is expensive (offering big discounts)
- When false positives annoy customers

**Your interpretation:**
"65.3% precision means our retention team won't waste too much time on customers who weren't actually leaving."

### **3. Recall (Sensitivity)**
```python
from sklearn.metrics import recall_score
recall = recall_score(y_test, y_pred)
# Your result: ~0.547 (54.7%)
```

**What it measures:** Of all actual churners, how many do you catch?
```
Recall = True Positives / (True Positives + False Negatives)
       = Caught Churners / All Actual Churners
```

**Business meaning:**
- You catch 55% of customers who would leave
- You miss 45% of churners

**When recall matters most:**
- When missing a churner is very costly
- When you have budget for broad retention campaigns

**Your interpretation:**
"We identify 54.7% of churners before they leave. While we miss some, this is much better than random guessing (26.5%)."

### **4. F1-Score**
```python
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred)
# Your result: ~0.595 (59.5%)
```

**What it measures:** Harmonic mean of precision and recall
```
F1 = 2 * (Precision * Recall) / (Precision + Recall)
```

**Why harmonic mean?**
- Penalizes extreme imbalances
- If either precision or recall is very low, F1 is low

**When to use:**
- When you need balance between precision and recall
- Good single metric for model comparison

**Your interpretation:**
"F1 of 0.595 shows reasonable balance - we're not sacrificing too much of either precision or recall."

---

## **Part 2: ROC-AUC - The Key Discriminator**

### **5. ROC-AUC Score**
```python
from sklearn.metrics import roc_auc_score, roc_curve

# Calculate
y_proba = pipeline.predict_proba(X_test)[:, 1]  # Get probabilities
roc_auc = roc_auc_score(y_test, y_proba)
# Your result: ~0.842
```

**What it measures:** 
How well the model ranks customers by churn probability

**Understanding the ROC Curve:**
```python
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
```
- **False Positive Rate (x-axis)**: Rate of false alarms at each threshold
- **True Positive Rate (y-axis)**: Rate of caught churners at each threshold
- **Each point**: Different probability threshold for classification

**Why AUC = 0.842 is good:**
- 0.5 = Random guessing (diagonal line)
- 1.0 = Perfect (impossible in practice)
- 0.842 = Strong discrimination ability

**Business interpretation:**
"If we randomly pick one churner and one non-churner, the model will rank the churner higher 84.2% of the time."

**Practical use:**
```python
# You can choose different thresholds based on business needs
# High threshold (e.g., 0.7) = High precision, low recall
# Low threshold (e.g., 0.3) = Low precision, high recall
```

---

## **Part 3: Precision-Recall Curve - Better for Imbalanced Data**

### **6. Average Precision Score**
```python
from sklearn.metrics import average_precision_score, precision_recall_curve

# Calculate
ap_score = average_precision_score(y_test, y_proba)
# Your result: ~0.651

# Baseline (random) would be:
baseline = y_test.mean()  # 0.265 (your churn rate)
```

**Why PR curve over ROC for imbalanced data:**
- ROC can be overly optimistic with imbalanced classes
- PR curve focuses on the minority class (churners)

**Interpretation:**
"Average precision of 0.651 vs baseline 0.265 means we're 2.45x better than random at identifying churners."

---

## **Part 4: Statistical Significance Tests**

### **7. Confusion Matrix Analysis**
```python
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

# Your typical results:
# TN: ~1001 (correctly identified non-churners)
# FP: ~35  (false alarms)
# FN: ~168 (missed churners)
# TP: ~205 (correctly identified churners)
```

**Statistical test for significance:**
```python
from scipy.stats import chi2_contingency

# Test if predictions are better than random
chi2, p_value, dof, expected = chi2_contingency(cm)

if p_value < 0.05:
    print("Model predictions are statistically significant (p < 0.05)")
```

### **8. Calibration Test**
```python
from sklearn.calibration import calibration_curve

# Check if predicted probabilities match actual frequencies
fraction_pos, mean_pred = calibration_curve(y_test, y_proba, n_bins=10)

# Perfect calibration: fraction_pos == mean_pred
# Plot these to see if your probabilities are reliable
```

**Why calibration matters:**
- If model says "70% chance of churn", do 70% actually churn?
- Important for risk-based decisions

---

## **Part 5: Business Metrics Translation**

### **Converting Statistical Metrics to Business Value**

```python
def calculate_business_metrics(cm, avg_customer_value=1500):
    """
    Translate confusion matrix to business metrics
    """
    tn, fp, fn, tp = cm.ravel()
    
    # Business calculations
    saved_customers = tp  # Churners we caught
    revenue_saved = saved_customers * avg_customer_value
    
    wasted_interventions = fp  # False alarms
    intervention_cost = wasted_interventions * 50  # $50 per intervention
    
    missed_opportunities = fn  # Churners we missed
    revenue_lost = missed_opportunities * avg_customer_value
    
    return {
        'revenue_saved': revenue_saved,
        'intervention_cost': intervention_cost,
        'revenue_lost': revenue_lost,
        'net_benefit': revenue_saved - intervention_cost
    }

# Apply to your results
business_metrics = calculate_business_metrics(cm)
print(f"Potential monthly savings: ${business_metrics['net_benefit']:,.0f}")
```

---

## **Choosing the Right Metric for Decisions**

### **Decision Framework**

```python
# Scenario 1: Limited retention budget
# → Optimize for PRECISION
# → Target only highest probability churners

# Scenario 2: High customer lifetime value
# → Optimize for RECALL
# → Cast wider net to catch more churners

# Scenario 3: Balanced approach
# → Optimize for F1-SCORE or ROC-AUC
# → Your current model choice
```

### **Practical Threshold Selection**

```python
# Instead of default 0.5 threshold, choose based on business needs
def find_optimal_threshold(y_test, y_proba, metric='f1'):
    """
    Find threshold that maximizes chosen metric
    """
    thresholds = np.arange(0.1, 0.9, 0.01)
    scores = []
    
    for thresh in thresholds:
        y_pred = (y_proba > thresh).astype(int)
        if metric == 'f1':
            score = f1_score(y_test, y_pred)
        elif metric == 'precision':
            score = precision_score(y_test, y_pred)
        elif metric == 'recall':
            score = recall_score(y_test, y_pred)
        scores.append(score)
    
    best_idx = np.argmax(scores)
    return thresholds[best_idx], scores[best_idx]

optimal_thresh, best_score = find_optimal_threshold(y_test, y_proba)
print(f"Optimal threshold: {optimal_thresh:.2f} with F1: {best_score:.3f}")
```

---

## **Summary: Your Model's Story in Statistics**

Your baseline model tells this statistical story:

1. **Overall Performance** (Accuracy: 79.6%) - Decent but not the full picture
2. **Business Precision** (65.3%) - 2 out of 3 flagged customers will actually churn
3. **Coverage** (Recall: 54.7%) - Catches half of all churners
4. **Discrimination** (ROC-AUC: 0.842) - Excellent at ranking customers by risk
5. **Practical Value** (AP: 0.651 vs 0.265 baseline) - 2.5x better than random

The key insight: Your model successfully identifies high-risk customers while maintaining reasonable precision, making it valuable for targeted retention campaigns. The 0.842 ROC-AUC indicates strong fundamental pattern recognition that can be refined in Level 4 with better features and models.